Notebooks for analysing BBH P-P test results.

Michael J. Williams 2023

In [ ]:
import glob
import re
import sys
import datetime

import bilby
import matplotlib.pyplot as plt
import numpy as np

basedir = "../../"
sys.path.append(basedir)

from pp_plot import make_pp_plot, cbc_param_labels as labels
from utils import configure_plotting, load_json, natural_sort

configure_plotting(basedir)
figsize = plt.rcParams['figure.figsize']
linewidth = 6.17804

Specify the result paths

In [ ]:
nessai_path = "outdir_nessai_fix/"
dynesty_path = "outdir_dynesty_precessing/"
inessai_path = "outdir_inessai_A/"

In [ ]:
def load_all_results(base_path, file_extension="hdf5"):
    path = f"{base_path}/result/*result.{file_extension}"
    files = natural_sort(glob.glob(path))
    print(f'Found: {len(files)} results')
    results = [bilby.core.result.read_in_result(f) for f in files]
    return results

In [ ]:
def load_nessai_result_files(base_path):
    nessai_path = f"{base_path}/result/*_nessai/result.json"
    nessai_files = natural_sort(glob.glob(nessai_path))
    print(f'Found: {len(nessai_files)} nessai results files')
    nessai_results = [load_json(f) for f in nessai_files]
    return nessai_results

In [ ]:
inessai_results = load_all_results(inessai_path)

In [ ]:
nessai_results = load_all_results(nessai_path)

In [ ]:
dynesty_results = load_all_results(dynesty_path)

In [ ]:
fig, _ = make_pp_plot(inessai_results, labels=labels, height=figsize[0], width=figsize[0])
fig.savefig(f"figures/pp_plot_inessai.pdf")

Smaller version of P-P plots for appendices

In [ ]:
for sampler, results in zip(
    ["nessai", "dynesty"],
    [nessai_results, dynesty_results]
):
    print(f"Sampler: {sampler}")
    fig, _ = make_pp_plot(results, labels=labels, height=0.65 * linewidth, width=0.65 * linewidth, legend_fontsize="small")
    fig.savefig(f"figures/pp_plot_{sampler}.pdf")

## Nessai result files

These files contain more statistics

In [ ]:
nessai_further_results = dict()

In [ ]:
nessai_further_results["nessai"] = load_nessai_result_files(nessai_path)

In [ ]:
nessai_further_results["inessai"] = load_nessai_result_files(inessai_path)

## Comparison

In [ ]:
all_results = dict(
    nessai=nessai_results,
    dynesty=dynesty_results,
    inessai=inessai_results,
)

In [ ]:
sampler_labels = {
    "dynesty": r"\textsc{dynesty}",
    "nessai": r"\textsc{nessai}",
    "inessai": r"\textsc{i-nessai}",
}

In [ ]:
summary = dict()
for sampler, results in all_results.items():
    wall_time = np.array([r.sampling_time.total_seconds() / 60 for r in results])
    likelihood_evaluations = np.array([r.num_likelihood_evaluations for r in results])
    summary[sampler] = dict(
        wall_time=wall_time,
        likelihood_evaluations=likelihood_evaluations,
    )

In [ ]:
def get_time(string):
    """Convert a time with days to seconds"""
    if "day" in string:
        days, string = string.split(", ")
        days = int(days.replace(" day", ""))
    else:
        days = None
    t = datetime.datetime.strptime(string, "%H:%M:%S.%f") - datetime.datetime(
        1900, 1, 1
    )
    if days:
        t += datetime.timedelta(days=days)
    return t.total_seconds()

In [ ]:
summary["nessai"]["wall_time"] = np.array([r["sampling_time"] / 60 for r in nessai_further_results["nessai"]])
summary["inessai"]["wall_time"] = np.array([(get_time(r["sampling_time"]) + r["redraw_time"]) / 60 for r in nessai_further_results["inessai"]])

In [ ]:
for sampler, result_summary in summary.items():
    print(sampler)
    print(f"Wall time: {np.median(result_summary['wall_time'])}")
    print(f"Likelihood evaluations: {np.median(result_summary['likelihood_evaluations'])}")

In [ ]:
for sampler in ["dynesty", "nessai"]:
    print(sampler)
    for key in ["wall_time", "likelihood_evaluations"]:
        print(f"ratio for {key}: {np.median(summary[sampler][key] / summary['inessai'][key])}")

In [ ]:
markers = {
    "inessai": ".",
    "nessai": "x",
    "dynesty": "+",
}
colours = {
    "inessai": "C0",
    "nessai": "C1",
    "dynesty": "C2",
}
size = {
    "inessai": 1,
    "nessai": 30,
    "dynesty": 30,
}
fig = plt.figure(figsize=(1.0 * figsize[0], 1.0 * figsize[1]))
plt.axvline(60, ls='--', label='1 hour', c='black', lw=1.0, zorder=-1)
plt.axvline(24 * 60, ls='--', label='24 hours', c='black', lw=1.0, zorder=-1)
plt.text(60, 6e7, '1 hour')
plt.text(24 * 60, 6e7, '24 hours')

for i, (sampler, results) in enumerate(summary.items()):
    plt.scatter(
        results["wall_time"],
        results["likelihood_evaluations"],
        marker=markers.get(sampler),
        label=sampler_labels.get(sampler),
        color=colours.get(sampler),
#         s=size.get(sampler),
    )
    
plt.xlabel('Time [minutes]')
plt.ylabel('Likelihood evaluations')
plt.xscale('log')
plt.yscale('log')
plt.tight_layout()
plt.grid()
fig.savefig("figures/bbh_comparison.pdf")